<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откроем-и-изучим-файл." data-toc-modified-id="Откроем-и-изучим-файл.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откроем и изучим файл.</a></span></li><li><span><a href="#Разбивка-данных-на-выборки." data-toc-modified-id="Разбивка-данных-на-выборки.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбивка данных на выборки.</a></span></li><li><span><a href="#Исследование-качества-разных-моделей." data-toc-modified-id="Исследование-качества-разных-моделей.-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование качества разных моделей.</a></span></li><li><span><a href="#Проверка-моделей-на-вменяемость." data-toc-modified-id="Проверка-моделей-на-вменяемость.-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка моделей на вменяемость.</a></span></li><li><span><a href="#Вывод-исследования" data-toc-modified-id="Вывод-исследования-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод исследования</a></span></li></ul></div>

# Рекомендация тарифов

# Описание Проекта

Оператор мобильной связи определил, что многие их клиенты не переходят на новые тарифы и продолжают пользоваться старыми тарифами. Нам необходимо создать модель которая будет способна правильно рекомендовать клиентами более подходящий для них тариф.

В нашем распоряжении есть информация о поведении клиентов, которые уже перешли на новые тарифы: "Смарт" и "Ультра".

Проведем исследование моделей и выберем наиболее точную. Точность модели должна быть более 75%
Так же проверим модели на вменяемость.

**Задача:**

Построить модель с максимально большим значением accuracy. **Минимальный порог точности модели 75%**

# Описание данных

- сalls — количество звонков
- minutes — суммарная длительность звонков в минутах
- messages — количество sms-сообщений
- mb_used — израсходованный интернет-трафик в Мб
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

## Откроем и изучим файл.

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
print(df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


In [3]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Описание данных**

сalls — количество звонков

minutes — суммарная длительность звонков в минутах

messages — количество sms-сообщений

mb_used — израсходованный интернет-трафик в Мб

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

In [4]:
df.isnull().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [5]:
df.duplicated().sum()

0

В данных пропусков и дубликатов нет. Можно приступать к разбивки на выборки

## Разбивка данных на выборки.

In [6]:
#Разделим данные на три выборки
train_valid, test = train_test_split(df, test_size=0.2,random_state=12345)
train, valid = train_test_split(train_valid, test_size=0.25,random_state=12345)

In [7]:
#Обозначим признаки и целевые признаки для выборок

#Признаки для тестовой обучающей выборки
features_train = train.drop(['is_ultra'], axis=1)
target_train = train['is_ultra']

#Признаки для валидационной выборки
features_valid = valid.drop(['is_ultra'], axis=1)
target_valid = valid['is_ultra']

#Признаки для тестовой выборки
features_test = test.drop(['is_ultra'], axis=1)
target_test = test['is_ultra']

In [8]:
display(features_train.shape)
display(features_valid.shape)
display(features_test.shape)

(1928, 4)

(643, 4)

(643, 4)

## Исследование качества разных моделей.

**Рассмотрим модели предсказаний и найдем наиболее точную**

- Обучим модель «Решающее дерево»

In [9]:
model_tree = DecisionTreeClassifier(random_state=12345)
model_tree.fit(features_train, target_train)

DecisionTreeClassifier(random_state=12345)

In [10]:
train_prediction_tree = model_tree.score(features_train, target_train)
valid_prediction_tree = model_tree.score(features_valid, target_valid)

print("Accuracy")
print(f"Обучающая выборка:{train_prediction_tree}")
print(f"Валидационная выборка:{valid_prediction_tree}")

Accuracy
Обучающая выборка:1.0
Валидационная выборка:0.7122861586314152


Модель переобучилась на тестовой выборке и не может дать высокой точности на валидационной выборке. Попробуем улучшить предсказания изменив гиперпараметры глубины максимум до 10 и узнаем улучшатся ли предсказания модели на валидационной выборке

In [11]:
best_model = None
best_result = 0
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model_tree.fit(features_train, target_train) 
    predictions_tree = model_tree.predict(features_valid) 
    result = accuracy_score(target_valid,predictions_tree)
    if result > best_result:
        best_model = model_tree
        best_result = result
print("Accuracy лучшей модели:", best_result)

Accuracy лучшей модели: 0.7744945567651633


Точность стала выше. Уточним какая глубина дала самую высокую точность

In [12]:
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions_tree = model_tree.predict(features_valid) 

    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid,predictions_tree)) 

max_depth = 1 : 0.7387247278382582
max_depth = 2 : 0.7573872472783826
max_depth = 3 : 0.7651632970451011
max_depth = 4 : 0.7636080870917574
max_depth = 5 : 0.7589424572317263
max_depth = 6 : 0.7573872472783826
max_depth = 7 : 0.7744945567651633
max_depth = 8 : 0.7667185069984448
max_depth = 9 : 0.7620528771384136
max_depth = 10 : 0.7713841368584758


Проверим лучшую модель на валидационной выборке с лучшими гиперпараметрами и убедимся в результате

In [13]:
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=7)
model_tree.fit(features_train,target_train)

train_prediction_tree = model_tree.score(features_train, target_train)
valid_prediction_tree = model_tree.score(features_valid, target_valid)

print(f"Accuracy обучающей выборки:{train_prediction_tree}")
print(f"Accuracy валидационной выборки:{valid_prediction_tree}")

Accuracy обучающей выборки:0.8506224066390041
Accuracy валидационной выборки:0.7744945567651633


- Обучим модель «Случайный лес» и проверим точность на валидационной выборке

In [14]:
#Обучим модель и найдем лучшую точность среди моделей случайного леса с числом деревьев от 1 до 50 c шагом 10
#С максимальной глубиной от 1 до 10


best_model_forest = None
best_result_forest = 0
best_est = 0
best_depth = 0
for est in range(10,51, 10):
    for depth in range (1, 11):
        model_rndforest = RandomForestClassifier(random_state=12345, n_estimators=est,max_depth=depth)
        model_rndforest.fit(features_train,target_train)
        predictions_rndforest = model_rndforest.predict(features_valid) 
        result_forest = accuracy_score(target_valid,predictions_rndforest) 
        if result_forest > best_result_forest:
            best_model_forest = model_rndforest
            best_result_forest = result_forest
            best_est = est
            best_depth = depth
print("Accuracy наилучшей модели на валидационной выборке:", best_result_forest
      ,"Количество деревьев:", best_est, "Максимальная глубина:", depth)

Accuracy наилучшей модели на валидационной выборке: 0.7978227060653188 Количество деревьев: 50 Максимальная глубина: 10


Проверим модель на валидационной выборке с лучшими гиперпараметрами и убедимся в результате

In [15]:
model_rndforest = RandomForestClassifier(random_state=12345, n_estimators=50,max_depth=10)
model_rndforest.fit(features_train,target_train)

train_prediction_rndforest = model_rndforest.score(features_train, target_train)
valid_prediction_rndforest = model_rndforest.score(features_valid, target_valid)

print(f"Accuracy обучающей выборки:{train_prediction_rndforest}")
print(f"Accuracy валидационной выборки:{valid_prediction_rndforest}")

Accuracy обучающей выборки:0.8947095435684648
Accuracy валидационной выборки:0.7978227060653188


- Обучим модель «Логистическая регрессия» и проверим точность

In [16]:
model_lregression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)

model_lregression.fit(features_train,target_train)
predictions_lregression = model_lregression.predict(features_valid)
result_lregression = accuracy_score(target_valid,predictions_lregression)
print(f"Accuracy на валидационной выборке:{result_lregression}")

Accuracy на валидационной выборке:0.7262830482115086


Посмотрим на разницу точности между обучающей и валидационной выборкой для этой модели

In [17]:
train_prediction_lregression = model_lregression.score(features_train, target_train)
valid_prediction_lregression = model_lregression.score(features_valid, target_valid)

print(f"Accuracy обучающей выборки:{train_prediction_lregression}")
print(f"Accuracy тестовой выборки:{valid_prediction_lregression}")

Accuracy обучающей выборки:0.7510373443983402
Accuracy тестовой выборки:0.7262830482115086


Соберем все значения проверок на валидационной выборке в одну таблицу

In [18]:
models_acc_score = {'DecisionTreeClassifier': valid_prediction_tree,
               'RandomForestClassifier': valid_prediction_rndforest, 
               'LogisticRegression': valid_prediction_lregression}
display('Accuracy тестовых выборок для всех моделей')
models_acc_score

'Accuracy тестовых выборок для всех моделей'

{'DecisionTreeClassifier': 0.7744945567651633,
 'RandomForestClassifier': 0.7978227060653188,
 'LogisticRegression': 0.7262830482115086}

Проведем проверку модели «Случайный лес» на тестовой выборке для модели, которая показала лучшие значения точности при проверке на валидационной выборке.

In [19]:
model_rndforest = RandomForestClassifier(random_state=12345, n_estimators=50,max_depth=10)
model_rndforest.fit(features_train,target_train)

test_prediction_rndforest = model_rndforest.score(features_test, target_test)

print(f"Accuracy тестовой выборки:{test_prediction_rndforest}")

Accuracy тестовой выборки:0.7993779160186625


Точность на тестовой выборке стала еще чуть выше, чем при проверке на валидационной. Это модель стала лучшей по итогу нашего исследования и будет выбрана рекомендуемой для помощи рекомендаций тарифов клиентам.

Стоит отметить что самый важный критерий для бизнеса - точность.Скорость работы модели так же важна и если модель делает предсказания слишком долго, то можно столкнуться с недовольством клинетов или специлистов которые пользуются этой моделью.

Отдельно для каждой модели:

- «Решающее дерево»          : Среднее качество и Высокая скорость
- «Случайный лес»            : Высокое качество и Средняя скорость
- «Логистическая регрессия»  : Низкое качество и Высокая скорость


При постановки задачи ничего не было сказано о скорости, поэтому этот критерий мы учитывать не будем.

##  Проверка моделей на вменяемость.

Для проверки моделей на вменяесть можно использовать клиссификатор DummyClassifier. Исходя из описания в документации этот класифиатор делает прогнозы, которые игнорируют входные признаки (features). Этот классификатор служит простой базой для сравнения с другими более сложными классификаторами.

Будем использовать гиперпараметры 'uniform' и 'most_frequent'

'uniform' - генерирует предсказания случайным образом.

'most_frequent' - всегда предсказывает наиболее часто встречающуюся метку в обучающем наборе.

In [20]:
dummy_model = DummyClassifier(random_state=12345, strategy='uniform')
dummy_model.fit(features_train, target_train)

dummy_valid = dummy_model.score(features_valid, target_valid)
dummy_test = dummy_model.score(features_test, target_test)

print(f'Результат случайной стратегии модели на валдационной выборке: {dummy_valid}')
print(f'Результат случайной стратегии модели на тестовой выборке: {dummy_test}')

Результат случайной стратегии модели на валдационной выборке: 0.5023328149300156
Результат случайной стратегии модели на тестовой выборке: 0.48989113530326595


In [22]:
dummy_model = DummyClassifier(random_state=12345, strategy='most_frequent')
dummy_model.fit(features_train, target_train)

dummy_valid = dummy_model.score(features_valid, target_valid)
dummy_test = dummy_model.score(features_test, target_test)

print(f'Результат случайной модели на валидационной выборке: {dummy_valid}')
print(f'Результат случайной модели на тестовой выборке: {dummy_test}')

Результат случайной модели на валидационной выборке: 0.6889580093312597
Результат случайной модели на тестовой выборке: 0.6951788491446346


Результат этих предсказаний дают ~50% для случайной стратегии гиперпараметров и не более ~70% для стретегии с предсказаниями наиболее часто встречающуюся метку в обучающем наборе. Это дает нам основание сделать вывовод, что мы доказали вменяемость моделей которые мы рассмотрели ранее.

## Вывод исследования

Изучив 3 модели и и проведя исследования на разных выборках были получены следующие значения точности для валидационных выборок:

- Модель «Решающее дерево»: 0.7744945567651633
- Модель «Случайного леса»: 0.7978227060653188
- Модель «Логистическая регрессия»: 0.7262830482115086

на основании этого модель «Случайного леса» была выбрана для проверки на тестовой выборке и показала значения точности:

- 0.7993779160186625

Модели «Решающее дерево» и «Случайного леса» показали более высокие значения точности прогнозов в сравнении с «Логистическая регрессия».

Если скорость обучения модели не является значительным праметром для определения лучшей модели, то модель «Случайного леса» мы определим как самую успешную по результатам исследования